In [157]:
import os
import yaml
import time
from tqdm import tqdm
from src.utils import batch_generate, tokens_generate, run_inference
from src.mem import check_memory
import importlib
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from dotenv import load_dotenv
import plotly.express as px
import torch
import torch.nn.functional as F
import huggingface
from datetime import datetime
load_dotenv('secrets.env')

with open("config/config.yaml", "r") as f: 
    config = yaml.safe_load(f)

model_str = 'deepseek_r1_qwendistill_1.5'
eval_df = 'big_bench'

ds = config['datasets'][eval_df]
model_path = config['models'][model_str]['path']

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

In [139]:
dataset = load_dataset(ds['source'], ds['subset']).shuffle(config['seed'])
tokenizer = AutoTokenizer.from_pretrained(model_path) 
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)

# Experiment
Goal: test model performance on 100 BBH questions w/ chain of thought reasoning =D

In [177]:
importlib.reload(src.utils)
import src.utils
help(model.generate)

Help on method generate in module transformers.generation.utils:

generate(
    inputs: Optional[torch.Tensor] = None,
    generation_config: Optional[transformers.generation.configuration_utils.GenerationConfig] = None,
    logits_processor: Optional[transformers.generation.logits_process.LogitsProcessorList] = None,
    stopping_criteria: Optional[transformers.generation.stopping_criteria.StoppingCriteriaList] = None,
    prefix_allowed_tokens_fn: Optional[Callable[[int, torch.Tensor], List[int]]] = None,
    synced_gpus: Optional[bool] = None,
    assistant_model: Optional[ForwardRef('PreTrainedModel')] = None,
    streamer: Optional[ForwardRef('BaseStreamer')] = None,
    negative_prompt_ids: Optional[torch.Tensor] = None,
    negative_prompt_attention_mask: Optional[torch.Tensor] = None,
    **kwargs
) -> Union[transformers.generation.utils.GenerateDecoderOnlyOutput, transformers.generation.utils.GenerateEncoderDecoderOutput, transformers.generation.utils.GenerateBeamDecoderOnlyOu

In [171]:
df = pd.DataFrame(dataset['train'][0:10])
df['input'] = apply_instruct_format(df['input'], model = model_str, is_math = True)

batches = batch_generate(df, ds['input_column'], ds['target_column'])
tokens = tokens_generate(batches, tokenizer, device = 'mps')
res = run_inference(model, tokens, tokenizer, time_tracking = True)

In [176]:

tokens = tokenizer(test[3], padding = True, truncation = True, return_tensors = 'pt').to(device)
response = model.generate(**tokens, temperature = 0.6, max_tokens = 2000)
full_response = tokenizer.decode(response[0], skip_special_tokens = True)

ValueError: The following `model_kwargs` are not used by the model: ['max_tokens'] (note: typos in the generate arguments will also show up in this list)

In [175]:
full_response.replace(test[3], '').strip() # nice! 

"Okay, so I'm trying to figure out whether the drunk driver caused the injury to Joe's son"